In [ ]:
import numpy as np
import math
import random 
import multiprocessing
import pandas as pd
import os

# ==========================================
# 核心逻辑移植：对应 DenSimu.R 的数据生成
# ==========================================
def generate_densimu_data_batch(n, m, seed):
    """
    Python implementation of the simulation logic from DenSimu.R
    """
    np.random.seed(seed)
    
    # 1. Generate Predictors X (9 dimensions as per DenSimu.R)
    X1 = np.random.uniform(-1, 0, n)
    X2 = np.random.uniform(0, 1, n)
    X3 = np.random.uniform(1, 2, n)
    X4 = np.random.normal(0, 1, n)
    X5 = np.random.normal(-10, 3, n)
    X6 = np.random.normal(10, 3, n)
    # p is probability of 0, 1. R: sample(c(0,1), n, TRUE, c(0.4, 0.6))
    X7 = np.random.choice([0, 1], n, p=[0.4, 0.6]) 
    X8 = np.random.choice([0, 1], n, p=[0.3, 0.7])
    X9 = np.random.choice([0, 1], n, p=[0.6, 0.4])

    # Stack features: shape (n, 9)
    X = np.stack([X1, X2, X3, X4, X5, X6, X7, X8, X9], axis=1)
    
    # Parameters from R code
    sigma0 = 3
    kappa = 1
    
    # Initialize output array
    # Shape: (n, m, x_dim) and (n, m)
    x_expanded = np.zeros((n, m, 9))
    y = np.zeros((n, m))
    
    for i in range(n):
        x_i = X[i]
        
        # 2. Calculate Expected Mean (Eta) and Sigma based on R formulas
        term1 = 3 * (np.sin(np.pi * x_i[0]) + np.cos(np.pi * x_i[1])) * x_i[7]
        term2 = (5 * (x_i[3]**2) + x_i[4]) * x_i[6]
        expect_eta_Z = term1 + term2
        
        term3 = 0.5 * (np.sin(np.pi * x_i[0]) + np.cos(np.pi * x_i[1])) * x_i[7]
        term4 = np.abs(5 * (x_i[3]**2) + x_i[4]) * x_i[6]
        expect_sigma_Z = sigma0 + term3 + term4
        
        # 3. Sample latent mean (mu) and sigma
        mu = np.random.normal(expect_eta_Z, 1.0) 
        
        # NumPy gamma uses shape (k) and scale (theta). 
        shape_param = (expect_sigma_Z**2) / kappa
        scale_param = kappa / expect_sigma_Z
        sigma = np.random.gamma(shape_param, scale_param)
        
        # 4. Generate m observations
        y_i = mu + sigma * np.random.normal(0, 1, m)
        
        y[i, :] = y_i
        x_expanded[i, :, :] = x_i 
        
    return x_expanded, y


def save_3d_data_to_csv(x_3d, y, filename, n, m, d):
    # 构建 ID 列
    sample_ids = np.repeat(np.arange(n), m)
    meas_ids = np.tile(np.arange(m), n)
    
    # 重塑 X 和 Y
    x_flat = x_3d.reshape(n * m, d)
    y_flat = y.reshape(n * m)
    
    # 合并
    data_matrix = np.column_stack((sample_ids, meas_ids, x_flat, y_flat))
    
    # 创建列名
    feature_columns = [f'feature_{i+1}' for i in range(d)]
    columns =  ['sample_id', 'measurement_id'] + feature_columns + ['target']
    
    # 保存
    df = pd.DataFrame(data_matrix, columns=columns)
    df['sample_id'] = df['sample_id'].astype(int)
    df['measurement_id'] = df['measurement_id'].astype(int)
    
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    df.to_csv(filename, index=False)


def savedata(seed):
    """
    Worker function
    """
    seed2 = ((seed+50) * 20000331)%2**31
    np.random.seed(seed2) 
    
    # 使用 visualize.py 中定义的 n_vector 保持一致
    n_train_values = [10, 13, 17, 23, 31, 42, 57, 78, 106, 145, 198, 271, 400]
    m_train_values = [20] # 固定 m=20
    d_feature = 9

    # 1. 生成测试集 (一次性生成)
    n_test = 10000
    m_test = 1 
    x_test, y_test = generate_densimu_data_batch(n_test, m_test, seed2)
    
    # 保存测试集
    x_test_flat = x_test[:, 0, :]
    y_test_flat = y_test[:, 0]
    
    test_df = pd.DataFrame(x_test_flat, columns=[f'feature_{i+1}' for i in range(d_feature)])
    test_df['target'] = y_test_flat
    os.makedirs("./data/test_data/", exist_ok=True)
    test_df.to_csv(f"./data/test_data/s{seed}.csv", index=False)
    
    # 2. 生成训练和验证集
    for n_train in n_train_values:
        for m_train in m_train_values:
            n_valid = math.ceil(n_train * 0.25)
            m_valid = m_train
            
            # 生成训练数据
            x, y = generate_densimu_data_batch(n_train, m_train, seed2)
            
            # 生成验证数据
            x_valid, y_valid = generate_densimu_data_batch(n_valid, m_valid, seed2 + 1)
            
            # 保存
            save_3d_data_to_csv(x, y, f"./data/train_data/n{n_train}m{m_train}s{seed}.csv",
                               n_train, m_train, d_feature)

            save_3d_data_to_csv(x_valid, y_valid, f"./data/valid_data/n{n_train}m{m_valid}s{seed}.csv",
                               n_valid, m_valid, d_feature)
    
    print(f"Seed {seed} completed.")

# ==========================================
# 主执行块
# ==========================================
seedlist = list(range(50)) 

if __name__ == '__main__':
    seeds = list(seedlist)
    nproc = 40 # 您的机器似乎核心数较多，可以适当调大
    
    # 【关键修改】移除 set_start_method('spawn')，Linux下默认使用 fork
    # 如果必须设置，使用: multiprocessing.set_start_method('fork', force=True)
    print("Starting data generation based on DenSimu.R logic...")
    
    with multiprocessing.Pool(processes = nproc) as pool: 
        pool.map(savedata, seedlist)
    print("All data generated.")

In [ ]:
import os, gc
import numpy as np
import math
import torch  
import torch.nn as nn     
from torch.utils.data import DataLoader, Dataset
import random 
import multiprocessing
import subprocess
import shutil
import pandas as pd
import time
from colorama import init, Fore

init(autoreset=True)

# 强制使用第一块 GPU (RTX 4090)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

class Args:
    def __init__(self, batch_size=10, lr =0.001, nepoch = 200, patience = 10, wide = 100, depth = 5, n_train=1, m_train=1) -> None:
        self.batch_size = batch_size
        self.lr = lr
        self.nepoch = nepoch 
        self.patience = patience 
        self.wide = wide 
        self.depth = depth 
        self.biaoji = "wide" + str(wide) + "depth" + str(depth) + "n" + str(n_train) + "m" + str(m_train)
        self.n_train = n_train
        self.m_train = m_train

class EarlyStopping():
    def __init__(self, save_path, args, verbose=False, delta=0):
        self.save_path = save_path 
        self.patience = args.patience 
        self.verbose = verbose 
        self.counter = 0 
        self.best_score = None 
        self.early_stop = False 
        self.val_loss_min = np.Inf 
        self.delta = delta 

    def __call__(self, model, train_loss, valid_loss, test_error, lipschitz, args, seed):
        score = -valid_loss 
        if self.best_score is None: 
            self.best_score = score 
            self.save_checkpoint(model, train_loss, valid_loss, test_error, lipschitz, args, seed) 
        elif score < self.best_score + self.delta: 
            self.counter += 1 
            if self.counter >= self.patience: 
                self.early_stop = True 
        else:
            self.best_score = score
            self.save_checkpoint(model, train_loss, valid_loss, test_error, lipschitz, args, seed)
            self.counter = 0

    def save_checkpoint(self, model, train_loss, valid_loss, test_error, lipschitz, args, seed):
        # 确保目录存在
        os.makedirs(self.save_path, exist_ok=True)
        torch.save(model.state_dict(), os.path.join(self.save_path, 'best' + str(seed) + args.biaoji +'network.pth') )
        torch.save(train_loss, os.path.join(self.save_path, 'best'+ str(seed) + args.biaoji +'train_loss.pth')) 
        torch.save(valid_loss, os.path.join(self.save_path, 'best'+ str(seed) + args.biaoji +'valid_loss.pth')) 
        torch.save(test_error, os.path.join(self.save_path, 'best'+ str(seed) + args.biaoji +'test_loss.pth')) 
        torch.save(lipschitz, os.path.join(self.save_path, 'best'+ str(seed) + args.biaoji +'lipschitz.pth')) 
        self.val_loss_min = valid_loss

class Dataset_repeatedmeasurement(Dataset): 
    def __init__(self, x, y) -> None:  
        super().__init__()
        self.x = x 
        self.y = y 
    def __len__(self) -> int: 
        return len(self.x) 
    def __getitem__(self, index): 
        return {"x" : self.x[index], "y" : self.y[index]}

class happynet(nn.Module):
    # 动态构建网络，保持你原有的逻辑
    def __init__(self, n_feature, n_hidden, n_output, n_layer): 
        super().__init__()
        layers = []
        # Input layer
        layers.append(nn.Linear(n_feature, n_hidden))
        layers.append(nn.ReLU())
        
        # Hidden layers (n_layer - 2)
        # Assuming n_layer includes input and output transitions conceptually in your original code logic
        # Original code: if n_layer=3 -> In(1)-Re-Hid(1)-Re-Out(1). Total linear layers = 3.
        for _ in range(n_layer - 2):
            layers.append(nn.Linear(n_hidden, n_hidden))
            layers.append(nn.ReLU())
            
        # Output layer
        layers.append(nn.Linear(n_hidden, n_output))
        
        self.net = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.net(x)

def compute_lipschitz_constant(model):
    """计算模型的全局Lipschitz常数（各层谱范数乘积）"""
    lipschitz = 1.0
    for layer in model.modules():
        if isinstance(layer, nn.Linear):
            W = layer.weight.detach()
            # 使用 float32 计算 SVD 可能会慢，但在 GPU 上通常很快
            try:
                # torch.linalg.svd 在某些 CUDA 版本可能不稳定，如果报错可转到 CPU
                _, s, _ = torch.linalg.svd(W, full_matrices=False)
                lipschitz *= s[0].item()
            except:
                # Fallback to CPU if CUDA SVD fails
                _, s, _ = torch.linalg.svd(W.cpu(), full_matrices=False)
                lipschitz *= s[0].item()
    return lipschitz

def load_model_and_compute_lipschitz(model_path, n_feature, n_hidden, n_output, n_layer, device):
    model = happynet(n_feature=n_feature, n_hidden=n_hidden, n_output=n_output, n_layer=n_layer).to(device)
    state_dict = torch.load(model_path, map_location=device)
    model.load_state_dict(state_dict)
    model.eval()
    lipschitz = compute_lipschitz_constant(model)
    return model, lipschitz

def GPUstrain(x, y, x_valid, y_valid, x_test, y_test, args, seed, device):
    # 自动获取特征维度
    x_dim = x.shape[1] 

    net = happynet(n_feature=x_dim, n_hidden=args.wide, n_output=1, n_layer=args.depth).to(device)
    
    optimizer = torch.optim.Adam(net.parameters(), lr=args.lr, betas=(0.90, 0.999), eps=1e-8) 
    loss_func = nn.MSELoss() 

    train_epochs_loss = [] 
    valid_epochs_loss = [] 
    test_epochs_error = [] 

    train_dataset = Dataset_repeatedmeasurement(x, y)
    train_dataloader = DataLoader(dataset=train_dataset, batch_size=args.batch_size, shuffle=True, pin_memory=True)

    # Move valid/test to GPU once
    x_valid = torch.from_numpy(x_valid).float().to(device) 
    y_valid = torch.from_numpy(y_valid).float().to(device) 
    x_test = torch.from_numpy(x_test).float().to(device)
    y_test = torch.from_numpy(y_test).float().to(device) 

    save_path = "./resultsv" 
    early_stopping = EarlyStopping(save_path, args=args)

    for epoch in range(args.nepoch): 
        net.train()
        train_epoch_loss = []

        for traindata in train_dataloader:
            x_train = traindata["x"].float().to(device, non_blocking=True)
            y_train = traindata["y"].float().to(device, non_blocking=True)
            
            outputs = net(x_train) 
            loss = loss_func(outputs.view(-1), y_train.view(-1))
            
            optimizer.zero_grad() 
            loss.backward() 
            optimizer.step() 
            
            train_epoch_loss.append(loss.item())

        avg_train_loss = np.mean(train_epoch_loss)        
        train_epochs_loss.append(avg_train_loss)

        # Validation
        with torch.no_grad():
            net.eval() 
            valid_predict = net(x_valid)
            loss_valid = loss_func(valid_predict.view(-1), y_valid.view(-1))
            valid_epochs_loss.append(loss_valid.item())

            test_predict = net(x_test)
            error_test = loss_func(test_predict.view(-1), y_test.view(-1))
            test_epochs_error.append(error_test.item())

            current_lipschitz = compute_lipschitz_constant(net)

        if epoch > 10 or args.n_train*args.m_train > 200:
            early_stopping(net, avg_train_loss, loss_valid.item(), error_test.item(), current_lipschitz, args, seed)
            if early_stopping.early_stop: 
                break

    return net, train_epochs_loss, valid_epochs_loss, test_epochs_error

def onedim(n_train, m_train, seed):
    # 设置设备
    device = torch.device("cuda:0") # 使用 4090

    seed2 = ((seed+50) * 20000331 )% 2**31
    torch.manual_seed(seed2) 
    np.random.seed(seed2) 
    random.seed(seed2) 
    
    n_valid = math.ceil(n_train*0.25)
    m_valid = m_train

    # 读取数据
    try:
        train_df = pd.read_csv(f"./data/train_data/n{n_train}m{m_train}s{seed}.csv")
        valid_df = pd.read_csv(f"./data/valid_data/n{n_train}m{m_valid}s{seed}.csv")
        test_df = pd.read_csv(f"./data/test_data/s{seed}.csv")
    except FileNotFoundError:
        print(f"Data not found for n={n_train}, m={m_train}, seed={seed}")
        return None

    # 提取特征列（自动过滤非feature列）
    feature_cols = [col for col in train_df.columns if col.startswith('feature_')]
    
    x_train = train_df[feature_cols].values
    y_train = train_df['target'].values
    x_valid = valid_df[feature_cols].values
    y_valid = valid_df['target'].values
    x_test = test_df[feature_cols].values
    y_test = test_df['target'].values

    # Batch size 策略
    total_samples = n_train * m_train
    if total_samples < 128:
       batch_size, lr = min(total_samples, 32), 0.0005
    elif total_samples < 1024:
        batch_size, lr = 64, 0.0005
    elif total_samples < 4096:
        batch_size, lr = 128, 0.001
    elif total_samples < 16384:
        batch_size, lr = 256, 0.002
    else:
        batch_size, lr = 1024, 0.002

    # 网络配置
    net_configs = [
        {'wide': 50, 'depth': 2, 'id': 0},
        {'wide': 100, 'depth': 3, 'id': 1},
        {'wide': 200, 'depth': 4, 'id': 2},
        {'wide': 400, 'depth': 5, 'id': 3},
        {'wide': 600, 'depth': 6, 'id': 4},
        {'wide': 800, 'depth': 6, 'id': 5}
    ]
    
    results = []
    
    for config in net_configs:
        args = Args(lr=lr, wide=config['wide'], depth=config['depth'], 
                   batch_size=batch_size, n_train=n_train, m_train=m_train)
        
        # 训练
        GPUstrain(x_train, y_train, x_valid, y_valid, x_test, y_test, args, seed2, device)
        
        # 加载最佳结果
        try:
            # 路径
            prefix = os.path.join('./resultsv', f'best{seed2}{args.biaoji}')
            train_loss = torch.load(prefix + 'train_loss.pth', map_location='cpu')
            valid_loss = torch.load(prefix + 'valid_loss.pth', map_location='cpu')
            test_loss = torch.load(prefix + 'test_loss.pth', map_location='cpu')
            lipschitz = torch.load(prefix + 'lipschitz.pth', map_location='cpu')
            
            results.append({
                'n_train': n_train,
                'm_train': m_train,
                'train_loss': float(train_loss),
                'valid_loss': float(valid_loss),
                'test_loss': float(test_loss),
                'lipschitz': float(lipschitz),
                'net_id': config['id']
            })
            
            # 清理模型文件以节省空间（可选）
            # os.remove(prefix + 'network.pth')
            
        except Exception as e:
            print(f"Error loading results for net {config['id']}: {e}")

    # 保存结果
    if results:
        df_res = pd.DataFrame(results)
        os.makedirs("./perf", exist_ok=True)
        df_res.to_csv(f'./perf/n{n_train}m{m_train}s{seed}.csv', index=False)
        return df_res
    return None

# ==========================================
# 主执行入口
# ==========================================
# 必须与数据生成和可视化保持一致
n_vector = [10, 13, 17, 23, 31, 42, 57, 78, 106, 145, 198, 271, 400]
m_vector = [20] # 固定采样频率
seeds = list(range(50)) # 50次重复实验

# 构建任务队列
tasks = []
for n in n_vector:
    for m in m_vector:
        for s in seeds:
            tasks.append((n, m, s))

if __name__ == '__main__': 
    multiprocessing.set_start_method('spawn', force=True) 
    
    # 针对 4090 单卡，不建议开启过多并行训练进程，因为显存和CUDA上下文切换开销大
    # 建议 nproc = 1 或 2。如果你的模型很小，可以尝试 4。
    # 这里设置为 4，利用 4090 的大显存（24GB）并行跑 4 个实验
    nproc = 4 
    
    print(f"Starting training on RTX 4090 with {nproc} parallel processes...")
    start_time = time.time()
    
    with multiprocessing.Pool(processes = nproc) as pool: 
        pool.starmap(onedim, tasks)
        
    print(f"Training completed in {time.time() - start_time:.2f} seconds.")

Starting training on RTX 4090 with 4 parallel processes...


Process SpawnPoolWorker-1:
Process SpawnPoolWorker-2:
Process SpawnPoolWorker-3:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/root/miniconda3/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/root/miniconda3/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/root/miniconda3/lib/python3.12/multiprocessing/pool.py", line 114, in worker
    task = get()
           ^^^^^
  File "/root/miniconda3/lib/python3.12/multiprocessing/queues.py", line 389, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'onedim' on <module '__main__' (<class '_frozen_importlib.BuiltinImporter'>)>
  File "/root/miniconda3/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/root/miniconda3/lib/python3.12/multiprocessing/process.py", li

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
import pandas as pd
import os

plt.rcParams["font.family"] = "Times New Roman"
plt.rc('font', size=15)
rc('text', usetex=False)

color_tuple = ['#ae1908', '#ec813b', '#05348b', '#9acdc4', '#00FF00', '#0000FF']
n_vector = [10, 13, 17, 23, 31, 42, 57, 78, 106, 145, 198, 271, 400]
fixed_m_train = 20

# 初始化存储矩阵
train_mse_matrix = np.zeros((len(n_vector), 6))
test_mse_matrix = np.zeros((len(n_vector), 6))
lip_matrix = np.zeros((len(n_vector), 6))

print("Loading data for visualization...")

for i, n_train in enumerate(n_vector):
    results = []
    for seed in range(50):
        file_path = f"./perf/n{n_train}m{fixed_m_train}s{seed}.csv"
        if os.path.exists(file_path):
            try:
                data = pd.read_csv(file_path)
                results.append(data)
            except Exception as e:
                print(f"Error reading {file_path}: {e}")
        else:
            # print(f"Missing: {file_path}") # Optional: reduce noise
            pass

    if results:
        all_results = pd.concat(results, ignore_index=True)
        # 按网络ID分组计算均值
        for net_id in range(6):
            net_data = all_results[all_results['net_id'] == net_id]
            if not net_data.empty:
                train_mse_matrix[i, net_id] = net_data['train_loss'].mean()
                test_mse_matrix[i, net_id] = net_data['test_loss'].mean()
                lip_matrix[i, net_id] = net_data['lipschitz'].mean()
    else:
        print(f"No valid data for n={n_train}")

# 绘图设置
lines = ['solid']*4 + ['dashed']*2
marker = ['o', 's', '^', 'v', 'D', 'x']
model_name = ['L=2, W=50','L=3, W=100', 'L=4, W=200', 'L=5, W=400', 'L=6, W=600', 'L=6, W=800']

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))
plt.subplots_adjust(top=0.95, bottom=0.15, left=0.1, right=0.95, wspace=0.3)

# Plot Training Error
for i in range(6):
    ax1.plot(n_vector, train_mse_matrix[:, i], color=color_tuple[i], linestyle=lines[i],
             label=model_name[i], marker=marker[i])
ax1.set_ylabel(r"$\mathtt{train-mse}$", fontsize=14)
ax1.set_xlabel(r"Training Set Size $n$", fontsize=14)
ax1.set_title(f"Training Error (m={fixed_m_train})", fontsize=14)
ax1.legend()

# Plot Test Error
for i in range(6):
    ax2.plot(n_vector, test_mse_matrix[:, i], color=color_tuple[i], linestyle=lines[i],
             label=model_name[i], marker=marker[i])
ax2.set_ylabel(r"$\mathtt{test-mse}$", fontsize=14)
ax2.set_xlabel(r"Training Set Size $n$", fontsize=14)
ax2.set_title(f"Test Error (m={fixed_m_train})", fontsize=14)
ax2.legend()

# Plot Lipschitz Constant
log_lip = np.log10(lip_matrix + 1e-10) # 加上 epsilon 防止 log(0)
for i in range(6):
    ax3.plot(n_vector, log_lip[:, i], color=color_tuple[i], linestyle=lines[i],
             label=model_name[i], marker=marker[i])
ax3.set_ylabel(r"Lipschitz constant $log_{10}(L)$", fontsize=14)
ax3.set_xlabel(r"Training Set Size $n$", fontsize=14)
ax3.set_title(f"Lipschitz Constant (m={fixed_m_train})", fontsize=14)
ax3.set_xscale("log")
ax3.legend()

save_path = './Simulation/Case1_R_Adapted'
os.makedirs(save_path, exist_ok=True)
plt.savefig(os.path.join(save_path, 'DenSimu_Result.png'), dpi=300, bbox_inches='tight')
print(f"Figure saved to {save_path}/DenSimu_Result.png")
# plt.show()